In [ ]:
import torch
import torchaudio
import matplotlib.pyplot as plt
from train import *
from dataset import *
from const import *
from tacotron_model import Tacotron2

In [ ]:
def get_output():
    loader = get_lj_loader(batch_size=4, limit=4)
    net = Tacotron2()
    net.to(device)
    net.load_state_dict(torch.load('data/ckpts/dotattn_latest.sd', map_location=torch.device('cpu')))
    net.eval()
    with torch.no_grad():
        for batch in loader:
            S_pad, S_lengths, token_pad, token_lengths = batch
            S_pad = S_pad.to(device)
            token_pad = token_pad.to(device)
            S_true = S_pad.cpu()
            S_before, S_after, stop_logits, attn_weights = net(token_pad, token_lengths, S_pad, teacher_forcing=True)
            tf_res = (S_before, S_after, stop_logits, attn_weights, S_true, S_lengths)
            S_before, S_after, stop_logits, attn_weights = net(token_pad, token_lengths, S_pad, teacher_forcing=False)
            infer_res = (S_before, S_after, stop_logits, attn_weights, S_true, S_lengths)
            break
    return tf_res, infer_res

In [ ]:
def check_output(res, check_idx=0):
    S_before, S_after, stop_logits, attn_weights, S_true, S_lengths = res
    S_len = S_lengths[check_idx].item()
    print('Target spectrogram')
    plt.imshow(S_true[:S_len, check_idx, :])
    plt.show()
    print('Output spectrogram')
    plt.imshow(S_after[:S_len, check_idx, :])
    plt.show()
    print('Stopper logits')
    #plt.plot(1.0/(1.0+torch.exp(-stop_logits[:, check_idx])))
    plt.plot(stop_logits[:, check_idx])
    plt.axvline(x=S_len, color='y')
    plt.axhline(y=0.0, color='y')
    print('two frames left: {:g}'.format(stop_logits[S_len-3, check_idx].item()))
    print('one frame left: {:g}'.format(stop_logits[S_len-2, check_idx].item()))
    print('should stop here: {:g}'.format(stop_logits[S_len-1, check_idx].item()))
    print('after zero frame: {:g}'.format(stop_logits[S_len, check_idx].item()))
    plt.show()
    print('Attn weights')
    plt.imshow(attn_weights[:S_len, :, check_idx])
    plt.show()

In [ ]:
tf_res, infer_res = get_output()

In [ ]:
check_output(tf_res, check_idx=0)

In [ ]:
check_output(infer_res, check_idx=0)